# 上下文管理器和else

with语句会设置一个临时的上下文，交给上下文管理其对象控制，并负责清理上下文。

for与else：
    仅当for循环运行完毕（即for循环没有被break语句中断）才运行else语句块
while与else：
    仅当while循环因为条件为假值而退出时（即while循环没有被break语句终止）才运行else语句块
    
在所有情况下，如果异常或者return，break，或continue语句导致控制权跳到了复合语句的主块之外，else子句也会被跳过。

In [1]:
try:
    dangerous_call()
except OSError:
    log('OSError ...')
else:
    after_call()

NameError: name 'dangerous_call' is not defined

只有try块不抛出异常，才会执行after_call()

In [16]:
class LookingGlass:
    
    # 除self之外，Python调用__enter__方法时不传入其他参数
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        # 为sys.stdout.write打猴子补丁
        sys.stdout.write = self.reverse_write
        # 返回一个字符串，这样才有内容存入目标变量
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
        
    # 如果一切正常，Python调用__exit__方法时传入的参数是None,None,None；如果抛出异常，这三个参数是异常数据
    # __exit__方法返回True，是为了告诉解释器异常已经处理了，此时解释器会压制异常，程序继续向后执行；如果__exit__方法返回None，或者True之外的值，with块中的任何异常将会向上冒泡
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero')
            return

In [17]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [18]:
what

'JABBERWOCKY'

解释器调用__enter__方法时,除了隐式的self之外，不会传入任何参数。

传给`__exit__`方法的三个参数：
1. `exc_type`:异常类（例如，`ZeroDivisionError`）
2. `exc_value`:异常实例。有时会有参数传给异常构造方法，例如错误消息，这些参数可以使用`exc_value.args`获取。
3. `traceback`:traceback对象。

In [19]:
manager = LookingGlass()
print(manager)

In [20]:
monster = manager.__enter__()

In [21]:
monster == 'JABBERWOCKY'

True

In [22]:
print(monster)

YKCOWREBBAJ


In [23]:
manager.__exit__(None, None, None)

In [24]:
print(monster)

JABBERWOCKY


## contextlib模块

### @contextmanager

在使用`@contextmanager`装饰器的生成器中，`yield`语句的作用是把函数的定义体分成两部分：`yield`语句前面的所有代码在`with`块开始时（即解释器调用`__enter__`方法时）执行，`yield`语句后面的代码在`with`块结束时（即调用`__exit__`方法时）执行。

In [1]:
import contextlib

@contextlib.contextmanager
def looking_glass_1():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write
    # 产出一个值，这个值会绑定到with语句中as子句的目标变量上。执行with块中的代码时，这个函数会在这一点暂停
    yield 'JABBERWOCKY'
    # 控制权一旦跳出with块，继续执行yield语句之后的代码
    sys.stdout.write = original_write

In [2]:
with looking_glass_1() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [3]:
print(what)

JABBERWOCKY


`@contextmanger`装饰器会把函数包装成实现`__enter__`和`__exit__`方法的类。

这个类的`__enter__`方法有如下的作用：

1. 调用生成器函数，保存生成器对象（这里把它称为gen）
2. 调用`next(gen)`，执行到`yield`关键字所在的位置
3. 返回`next(gen)`产出的值，以便把产出的值绑定到`with/as`语句中的目标变量上

`with`块终止时，`__exit__`方法会做以下几件事：

1. 检查有没有把异常传给`exc_type`；如果有，调用`gen.throw(exception)`，在生成器函数定义体中包含`yield`关键字的那一行抛出异常
2. 否则，调用`next(gen)`，继续执行生成器函数定义体中`yield`语句之后的代码

`with`块终止时，如果出现异常，不会执行`yield`语句之后的代码，可能是不安全的。就本例而言，如果`with`语句发生异常，那么`sys.stdout.write`将永远无法恢复成原来的状态。

In [4]:
@contextlib.contextmanager
def looking_glass_2():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT division by zero!'
    # 即使发生异常，依然能做一些收尾工作
    finally:
        sys.stdout.write = original_write
        if msg: print(msg)

`@contextlib.contextmanager`装饰器提供的`__exit__`方法默认行为是：假定发给生成器的所有异常都得到了处理，因此会对异常进行压制（进行压制的意思是不会向上冒泡异常）。如果不想`@contextmanager`压制异常，必须在被装饰的函数中显式重新抛出异常。

**重点：使用`@contextmanager`装饰器时，要把`yield`语句放在`try/finally`语句中。**

在`@contextmanager`装饰器的生成器中，`yield`与迭代没有任何关系，在这里生成器函数的作用更像协程：执行到某一点时暂停，让客户代码运行，直到客户让协程继续做事。

`with`语句块中发生异常的语句之后的代码不会执行。

In [17]:
with looking_glass_2() as what:
    print('ABCDEFG')
    print('='*20, ' reset sys.stdout.write '[::-1], '='*20)
    print(what)
    
print('ssssdff')

GFEDCBA
====================  reset sys.stdout.write  ====================
YKCOWREBBAJ
ssssdff


In [18]:
with looking_glass_2() as what:
    print('ABCDEFG')
    raise ZeroDivisionError
    # 发生异常之后的代码不会被执行
    print('='*20, ' reset sys.stdout.write '[::-1], '='*20)
    print(what)
    
print('ssssdff')

GFEDCBA
Please DO NOT division by zero!
ssssdff


In [16]:
with looking_glass_1() as what:
    print('ABCDEFG')
    raise ZeroDivisionError
    print(what)
    
print('ssssdff')

ABCDEFG


ZeroDivisionError: 

总而言之：`@contextmanger`装饰器能把一个包含`yield`语句的简单生成器变成上下文管理器———这比定义一个至少包含两种方法的类要简洁。